In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 14.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import pandas as pd
import numpy as np

import time
import pickle


from datasets import load_dataset
from datasets import load_metric

from sklearn.model_selection import train_test_split

from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import Trainer

from transformers import AutoModelForSequenceClassification


In [3]:
dataset=load_dataset("dair-ai/emotion")
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/dair-ai___emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

The dataset already has labels for the classes.

In [4]:
tokenizer=AutoTokenizer.from_pretrained('bert-base-cased')

In [5]:
def tokenize_data(example):
    return tokenizer(example['text'], padding='max_length')

dataset = dataset.map(tokenize_data, batched=True)



Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [7]:
remove_columns = ['text']
dataset = dataset.map(remove_columns=remove_columns)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [8]:
training_args = TrainingArguments("test_trainer", num_train_epochs=3)

In [9]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=6)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [11]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=dataset['train'], eval_dataset=dataset['validation'])

In [12]:
%%time
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6000
  Number of trainable parameters = 108314886


Step,Training Loss
500,0.733600
1000,0.387100
1500,0.271000
2000,0.275600
2500,0.186200
3000,0.176900
3500,0.163600
4000,0.177400
4500,0.121100
5000,0.127600


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-30

CPU times: user 1h 7min 28s, sys: 7min 15s, total: 1h 14min 43s
Wall time: 1h 15min 56s


TrainOutput(global_step=6000, training_loss=0.23688361676534017, metrics={'train_runtime': 4556.9356, 'train_samples_per_second': 10.533, 'train_steps_per_second': 1.317, 'total_flos': 1.2629784231936e+16, 'train_loss': 0.23688361676534017, 'epoch': 3.0})

In [13]:
trainer.save_model("bert_finetuned_model2")

Saving model checkpoint to bert_finetuned_model2
Configuration saved in bert_finetuned_model2/config.json
Model weights saved in bert_finetuned_model2/pytorch_model.bin


In [15]:
#metric = load_metric("f1_score")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    return acc

trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8


{'eval_loss': 0.1658184826374054,
 'eval_runtime': 60.5965,
 'eval_samples_per_second': 33.005,
 'eval_steps_per_second': 4.126,
 'epoch': 3.0}

In [16]:
predicted_results = trainer.predict(dataset['test'])

***** Running Prediction *****
  Num examples = 2000
  Batch size = 8


In [17]:
predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
#predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

In [18]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report


In [19]:
print(classification_report(dataset['test']['label'], 
                            predicted_labels))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       581
           1       0.96      0.95      0.95       695
           2       0.84      0.88      0.86       159
           3       0.97      0.90      0.94       275
           4       0.87      0.92      0.90       224
           5       0.78      0.76      0.77        66

    accuracy                           0.94      2000
   macro avg       0.90      0.90      0.90      2000
weighted avg       0.94      0.94      0.94      2000

